# 0.라이브러리 로딩

In [1]:
import os
import librosa
import torch
from logger import utils
from tqdm import tqdm
from glob import glob
from pydub import AudioSegment
from logger.utils import traverse_dir

# Cuda setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# configure loading
args = utils.load_config('./configs/sins.yaml')

# set path
MP4_DATA_PATH   = 'preprocess/mp4'
ORIGINAL_PATH   = 'preprocess/original/'
DEMUCS_PATH     = 'preprocess/demucs/'
NORM_PATH       = 'preprocess/norm/'
TEMP_LOG_PATH   = 'temp_ffmpeg_log.txt'  # ffmpeg의 무음 감지 로그의 임시 저장 위치

In [23]:
print(device)

cpu


# 1.데이터 전처리
***
1. 난 전처리가 필요없다. 배경음 제거가 완벽하고, 모든 데이터들도 특정 길이로 잘 잘려져있다.
    - 데이터를 전부 data/train/audio/안에 다 집어 넣고
        ```
        # training dataset
        data/train/audio/aaa.wav
        data/train/audio/bbb.wav
        ...
        ```
    - 1.6 validation 분리단계로 점프
***
2. 난 거의 다 되어 있지만 데이터가 너무 길다. 특정 길이로 자르고 싶다.
    - 데이터를 전부 preprocess/split 안에 다 집어 넣고
        ```
        # training dataset
        preprocess/norm/aaa.wav
        preprocess/norm/bbb.wav
        ...
        ```
    - 1.4 split 단계로 점프
***
3. 난 배경음도 제거해야되고 데이터도 길다. 거의 날 것의 상태다.
    - 데이터를 전부 preprocess/original 안에 다 집어넣고
        ```
        # training dataset
        preprocess/original/aaa.wav
        preprocess/original/bbb.wav
        ...
        ```
    - 1.2부터 demucs 단계로 점프
***
4. 난 아무것도 안되어 있고, 심지어 mp4파일이다.
    - 데이터들 전부 preprocess/mp4에 다 집어넣고
        ```
        # training dataset
        preprocess/mp4/aaa.mp4
        preprocess/mp4/bbb.mp4
        ...
        ```
    - 1.1부터 차례대로 진행

## 1.1데이터가 mp4인 경우 (wav만 있는 경우에는 패스)
preprocess/mp4 안에 있는 mp4파일을 wav로 변경 해서 preprocess/original 에 저장

In [2]:
def mp4_to_wav(input_dir : str, input_file: str, output_dir: str):
    """mp4파일을 wav형식으로 변환합니다.
    Args:
        input_dir (str) : 입력 mp4파일의 path
        input_file (str) : 입력 mp4파일의 이름
        output_dir (str) : 출력 wav파일의 path
    """
    ext = os.path.splitext(input_file)[1][1:]

    if ext != "mp4":
        return 
    else :
        track = AudioSegment.from_file(os.path.join(input_dir,input_file),  format= 'mp4')
        track = track.set_frame_rate(44100)
        track.export(os.path.join(output_dir, input_file[:-4]+".wav"), format='wav')


filelist =  traverse_dir(
    MP4_DATA_PATH,
    extension='mp4',
    is_pure=True,
    is_sort=True,
    is_ext=True)

for file in tqdm(filelist):
    mp4_to_wav(MP4_DATA_PATH, file, ORIGINAL_PATH)

100%|██████████| 12/12 [00:04<00:00,  2.63it/s]


## 1.2 무음제거 (Demucs)
preprocess/original에 있는 wav파일들의 음악소리를 제거하고 목소리만 추출해서 preprocess/demucs 에 저장

In [3]:
from sep_wav import demucs

demucs(ORIGINAL_PATH, DEMUCS_PATH)

2023-06-16 10:33:16 | INFO | torchaudio.utils.download | The local file (C:\Users\NT550-048\.cache\torch\hub\torchaudio\models\hdemucs_high_trained.pt) exists. Skipping the download.


Sample rate: 44100


목소리 추출 중...: 100%|██████████| 67/67 [2:54:42<00:00, 156.46s/it]  


## 1.3 normalize
preprocess/demucs에 있는 배경음이 제거된 데이터들을 노멀라이즈 (sample rate값을 바꿀 수 있음) 해서 preprocess/norm에 저장

In [4]:
from sep_wav import audio_norm

for filepath in tqdm(glob(DEMUCS_PATH+"*.wav"), desc="노멀라이징 작업 중..."):
    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(NORM_PATH, filename) + ".wav"
    audio_norm(filepath, out_filepath, sample_rate = 44100)

노멀라이징 작업 중...:   0%|          | 0/67 [00:00<?, ?it/s]

노멀라이징 작업 중...: 100%|██████████| 67/67 [04:13<00:00,  3.78s/it]


## 1.4 split
preprocess/norm에 있는 노말라이즈된 데이터들을 15초 길이로 잘라서 data/train/audio에 저장

In [5]:
import subprocess

for filepath in tqdm(glob(NORM_PATH+"*.wav"), desc="음원 자르는 중..."):
    duration = librosa.get_duration(filename=filepath)
    max_last_seg_duration = 0
    sep_duration_final = 15
    sep_duration = 15

    while sep_duration > 4:
        last_seg_duration = duration % sep_duration
        if max_last_seg_duration < last_seg_duration:
            max_last_seg_duration = last_seg_duration
            sep_duration_final = sep_duration
        sep_duration -= 1

    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(args.data.train_path,"audio", f"{filename}-%04d.wav")
    subprocess.run(f'ffmpeg -i "{filepath}" -f segment -segment_time {sep_duration_final} "{out_filepath}" -y', capture_output=True, shell=True)


음원 자르는 중...: 100%|██████████| 67/67 [00:28<00:00,  2.36it/s]


## 1.5 무음 제거
data/train/audio에 있는 잘라진 음원들 중에 무음인 파일들을 제거

In [6]:
from sep_wav import get_ffmpeg_args
import subprocess

for filepath in tqdm(glob(args.data.train_path+"/audio/*.wav"), desc="무음 제거 중..."):
    if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

    ffmpeg_arg = get_ffmpeg_args(filepath)
    subprocess.run(ffmpeg_arg, capture_output=True, shell=True)

    start = None
    end = None

    with open(TEMP_LOG_PATH, "r", encoding="utf-8") as f:
        for line in f.readlines():
            line = line.strip()
            if "lavfi.silence_start" in line:
                start = float(line.split("=")[1])
            if "lavfi.silence_end" in line:
                end = float(line.split("=")[1])

    if start != None:
        if start == 0 and end == None:
            os.remove(filepath)
            
if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

무음 제거 중...: 100%|██████████| 2331/2331 [02:00<00:00, 19.37it/s]


## 1.6 학습 데이터 중, 일부를 validaion용으로 자동으로 보내준다.
- data/train/audio에 있는 데이터 중 일정 비율만큼 알아서 data/val/audio로 이동시켜준다
    - 계산식은 다음과 같다 `max(2, min(10, 전체 데이터 * 0.01))`

In [7]:
from draw import main

In [8]:
from draw import main

main()

100%|██████████| 10/10 [00:00<00:00, 1110.16it/s]


## 2. 데이터 전처리 (학습에 쓰기 위한)

In [9]:
from preprocess import preprocess
from ddsp.vocoder import F0_Extractor, Volume_Extractor, Units_Encoder
from diffusion.vocoder import Vocoder

In [10]:
from preprocess import preprocess
from ddsp.vocoder import F0_Extractor, Volume_Extractor, Units_Encoder
from diffusion.vocoder import Vocoder

# get data
sample_rate = args.data.sampling_rate
hop_size = args.data.block_size

# initialize f0 extractor
f0_extractor = F0_Extractor(
                    args.data.f0_extractor, 
                    args.data.sampling_rate, 
                    args.data.block_size, 
                    args.data.f0_min, 
                    args.data.f0_max)

# initialize volume extractor
volume_extractor = Volume_Extractor(args.data.block_size)

# initialize mel extractor
mel_extractor = None
if args.model.type == 'Diffusion':
    mel_extractor = Vocoder(args.vocoder.type, args.vocoder.ckpt, device = device)
    if mel_extractor.vocoder_sample_rate != sample_rate or mel_extractor.vocoder_hop_size != hop_size:
        mel_extractor = None
        print('Unmatch vocoder parameters, mel extraction is ignored!')

# initialize units encoder
if args.data.encoder == 'cnhubertsoftfish':
    cnhubertsoft_gate = args.data.cnhubertsoft_gate
else:
    cnhubertsoft_gate = 10             
units_encoder = Units_Encoder(
                    args.data.encoder, 
                    args.data.encoder_ckpt, 
                    args.data.encoder_sample_rate, 
                    args.data.encoder_hop_size, 
                    device = device)    

# preprocess training set
preprocess(args.data.train_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device)

# preprocess validation set
preprocess(args.data.valid_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device)

 [Encoder Model] HuBERT Soft
 [Loading] pretrain/hubert/hubert-soft-0d54a1f4.pt
Preprocess the audio clips in : data/train\audio


  5%|▍         | 97/2089 [02:37<55:48,  1.68s/it]  


[Error] F0 extraction failed: data/train\audio\07 김광석 (Kim Kwang Seok) - 기다려줘 (Please Wait for Me) (Official Audio) (2022 Remastered)-0000.wav
This file has been moved to data/train\skip\07 김광석 (Kim Kwang Seok) - 기다려줘 (Please Wait for Me) (Official Audio) (2022 Remastered)-0000.wav


 17%|█▋        | 353/2089 [09:48<49:06,  1.70s/it]  


[Error] F0 extraction failed: data/train\audio\12 #### 김광석 (Kim Kwang Seok) - 1집 (First Album) (Full Track) (Official Audio) (2022 Remastered)-0177.wav
This file has been moved to data/train\skip\12 #### 김광석 (Kim Kwang Seok) - 1집 (First Album) (Full Track) (Official Audio) (2022 Remastered)-0177.wav


...

 92%|█████████▏| 1928/2089 [55:58<04:23,  1.64s/it]


[Error] F0 extraction failed: data/train\audio\67 #### 김광석 (Kim Kwang Seok) - 다시 부르기 2 (Sing Again 2) (Full Track) (Official Audio) (2022 Remastered)-0031.wav
This file has been moved to data/train\skip\67 #### 김광석 (Kim Kwang Seok) - 다시 부르기 2 (Sing Again 2) (Full Track) (Official Audio) (2022 Remastered)-0031.wav


100%|██████████| 2089/2089 [1:00:31<00:00,  1.74s/it]


Preprocess the audio clips in : data/val\audio


100%|██████████| 10/10 [00:17<00:00,  1.72s/it]


## 3. 학습

In [22]:
from train import ddsp_train

ddsp_train(args)

 >    exp: exp/sins-test
 [DDSP Model] Sinusoids Additive Synthesiser
 [*] restoring model from exp/sins-test\model_42000.pt
Load all the data from : data/train


100%|██████████| 2084/2084 [01:32<00:00, 22.56it/s]


Load all the data from : data/val


100%|██████████| 10/10 [00:00<00:00, 20.69it/s]


--- model size ---
model: 3,375,360
======= start training =======
epoch: 0 |   9/ 87 | exp/sins-test | batch/s: 0.09 | loss: 1.262 | time: 0:01:49.5 | step: 42010
epoch: 0 |  19/ 87 | exp/sins-test | batch/s: 0.13 | loss: 1.159 | time: 0:03:05.2 | step: 42020
epoch: 0 |  29/ 87 | exp/sins-test | batch/s: 0.14 | loss: 1.235 | time: 0:04:15.5 | step: 42030
epoch: 0 |  39/ 87 | exp/sins-test | batch/s: 0.14 | loss: 1.222 | time: 0:05:28.7 | step: 42040
epoch: 0 |  49/ 87 | exp/sins-test | batch/s: 0.14 | loss: 1.103 | time: 0:06:41.5 | step: 42050
epoch: 0 |  59/ 87 | exp/sins-test | batch/s: 0.14 | loss: 1.142 | time: 0:07:51.2 | step: 42060
epoch: 0 |  69/ 87 | exp/sins-test | batch/s: 0.11 | loss: 1.211 | time: 0:09:22.9 | step: 42070
epoch: 0 |  79/ 87 | exp/sins-test | batch/s: 0.12 | loss: 1.223 | time: 0:10:43.1 | step: 42080
epoch: 1 |   2/ 87 | exp/sins-test | batch/s: 0.14 | loss: 1.159 | time: 0:11:56.2 | step: 42090
epoch: 1 |  12/ 87 | exp/sins-test | batch/s: 0.10 | loss: 1

## 4. 결과물 뽑기

In [11]:
from types import SimpleNamespace
from main import inference

In [21]:

# configure setting
configures = {
    'model_path'            :   "exp/sins-test/model_42000.pt", # 추론에 사용하고자 하는 모델, 바로위에서 학습한 모델을 가져오면댐
    'input'                 :   "data/test/천년의사랑_vocals.wav", # 추론하고자 하는 노래파일의 위치 - 님들이 바꿔야댐 
    'output'                :   'data/김광석_ddsp_천년의사랑_42000.wav',  # 결과물 파일의 위치
    'device'                :   'cpu',
    'spk_id'                :   '1', 
    'spk_mix_dict'          :   'None', 
    'key'                   :   '0', 
    'enhance'               :   'true' , 
    'pitch_extractor'       :   'crepe' ,
    'f0_min'                :   '50' ,
    'f0_max'                :   '1100',
    'threhold'              :   '-60',
    'enhancer_adaptive_key' :   '0'
}
cmd = SimpleNamespace(**configures)

inference(cmd)

 [DDSP Model] Sinusoids Additive Synthesiser
 [Loading] exp/sins-test/model_42000.pt
MD5: 29858198b44cde1695beaffe7fecff7a
Loading pitch curves for input audio from cache directory...
Extracting the volume envelope of the input audio...
 [Encoder Model] HuBERT Soft
 [Loading] pretrain/hubert/hubert-soft-0d54a1f4.pt
Enhancer type: nsf-hifigan
| Load HifiGAN:  pretrain/nsf_hifigan/model
Removing weight norm...
Speaker ID: 1
Cut the input audio into 23 slices


100%|██████████| 23/23 [03:39<00:00,  9.53s/it]
